## Comment letters

Read the comment letters, compute doc2vec vector (for each letter) and group similar letters in 10 buckets

See https://towardsdatascience.com/automatic-topic-clustering-using-doc2vec-e1cea88449c
and https://github.com/olafmaas/hackdelft/blob/master/hackdelft/doc2vec/generate_json.py            

# make df and iterate

In [1]:
# all imports
import os as os
import pandas as pd
import glob
import csv
from pathlib import Path
import html, re
from w3lib.html import replace_entities
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [2]:
# matching filenames

with open ('/blue/acg7849/share/BS/summary.txt') as a: 
    summary = a. readlines()
print (summary[0:5])

sumdf = pd.DataFrame (summary)
#sumdf.str.split(pat="|")

new = sumdf[0].str.split("|", expand = True)
sumdf["CIK"]= new[0]
sumdf["coName"]= new[1]
sumdf["formtype"]= new[2]
sumdf["date"]= new[3]
sumdf["fName"]= new[4]
sumdf["length"]= new[5]
sumdf = sumdf.drop(columns=[0])
sumdf = sumdf.drop([0])

#print (sumdf[0:15])                        
#Initialize csv - no clusters
#sumdf.to_csv('ASG4.csv', index = True)

['CIK|coname|formtype|date|filename|length\n', '0001289850|NeuroMetrix, Inc.|10-K|20171231|267762.txt|132616\n', '0001345016|YELP INC|10-K|20191231|280603.txt|152323\n', '0001245791|GI DYNAMICS, INC.|10-K|20171231|267435.txt|28236\n', '0001772177|KURA SUSHI USA, INC.|10-K|20190831|275323.txt|38468\n']


In [3]:
sumdf["length"]= pd.to_numeric(sumdf["length"])
# filter out all the things with <1000 length 
sumdf = sumdf.loc[sumdf["length"] >= 1000]
#print (sumdf[0:15])     # seems filtering success! 

sumdf_bk  = sumdf  # make another file as backup
sumdf_test  = sumdf[:100]  # test length = 100

mydirectory = '/blue/acg7849/hli1/item1'
newdirectory = '/blue/acg7849/share/BS/item1'

sumdf["full_link"]= newdirectory +'/'+ sumdf["fName"]
#print (sumdf[0:13]) 
listoflink = list(sumdf["full_link"]) # in case we need a list
listofmylink = list(mydirectory +'/'+ sumdf["fName"])

print (listofmylink[0:5])

['/blue/acg7849/hli1/item1/267762.txt', '/blue/acg7849/hli1/item1/280603.txt', '/blue/acg7849/hli1/item1/267435.txt', '/blue/acg7849/hli1/item1/275323.txt', '/blue/acg7849/hli1/item1/277539.txt']


In [4]:
import gensim
import os, string, re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english') )

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

/apps/jupyterhub/1.1.0/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [28]:
# add some punctuation to string.punctuation
punc = string.punctuation + '“”'

# documents get tagged by an index (nu,mber), while filenames have different numbers
# keep track of this
fileIdToIndex = {} # given a fileId -> tag
indexToFileId=[] # given a tag -> fileId

class MyCommentLetters(object):
    def __init__(self, dirname, tokens_only = False):
        self.dirname = dirname
        self.tokens_only = tokens_only
 
    def __iter__(self):
        for i, fname in enumerate(os.listdir(self.dirname)[0:300]):  # here are the number of files at max?
        #for fname in os.listdir(self.dirname):
            with open( os.path.join(self.dirname, fname), encoding='utf-8') as f:
                content = f.read()
            
            # grab id from filename
            myCounter = int (  re.findall(r'(\d*)\.txt', fname)[0] )
            # update 
            fileIdToIndex [ myCounter] = i
            indexToFileId.append( myCounter)
            #print('fname', fname, 'tag', myCounter)
            file_tokens = [x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation]
            
            if self.tokens_only == True:
                yield file_tokens
            else:
                yield TaggedDocument(words=file_tokens, tags=[i] )                    

In [29]:
# make a file address in my HPG directory, write from listoflink
# copy and paste
#listoflink
import shutil

# open one file and then write it to directory

for i in listoflink:
    for p in listofmylink:
        with open( i, "r", encoding='utf-8') as f:
            output = f.read()
        with open( p, "r", encoding='utf-8') as f:
            mine = f.write()

            myfile = open(filename, 'w')
            myfile.writelines(s)
            myfile.close()

FileNotFoundError: [Errno 2] No such file or directory: '/blue/acg7849/hli1/item1/267762.txt'

In [30]:
# Hipergator
wordLists = MyCommentLetters(newdirectory) # a memory-friendly iterator, should iterate through a file address

In [31]:
# create a model, build vocabulary
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=10)
model.build_vocab(wordLists)   # list index out of range

In [9]:
#fileIdToIndex

In [33]:
indexToFileId[ 290  ]

267106

In [34]:
myText = '10-K'
print( myText.isalpha() )

False


In [35]:
# train it
model.train(wordLists, total_examples=model.corpus_count, epochs=model.epochs)

In [36]:
# because we are using a generator, the files are no longer in memory
# Hipergator
def tokenizeFile(file_id):
    with open( r'/blue/acg7849/share/BS/item1/'+str(file_id)+'.txt', encoding='utf-8') as f:
            content = f.read()
    return ([x for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation] )

In [37]:
t = tokenizeFile(267762)
model.infer_vector( t )

array([-1.196132  , -5.127264  , -3.1189978 ,  0.40635327, -0.6051856 ,
       -1.0871681 , -3.1914911 , -4.4955297 , -5.23893   , -1.9187999 ,
        1.250936  ,  2.6159403 , -4.534772  ,  3.9957583 , -3.7161956 ,
       -0.5586631 ,  1.2324674 , -0.53410465, -3.0927725 , -2.7281384 ,
       -1.1377373 , -0.4621333 ,  0.9403881 , -0.4959484 ,  4.9005747 ,
       -0.73659253,  2.1769545 ,  3.2041395 , -0.71954346,  0.33247924,
       -2.496773  ,  1.8453736 , -2.8237495 , -0.9597754 ,  1.0134658 ,
       -2.6949205 ,  1.4732404 ,  0.55403125, -0.44319275,  0.6034303 ,
       -2.5383945 ,  1.1881021 ,  0.5423492 , -1.0921609 ,  0.6440071 ,
        0.6952304 , -3.2677782 ,  1.213193  ,  1.7657232 , -1.062602  ,
       -0.41510707, -0.28982353, -2.104023  , -3.7741446 , -0.2790829 ,
        2.8520324 ,  0.33070278,  2.0624163 ,  2.5022595 ,  3.0887873 ,
       -1.5978309 ,  1.5314784 ,  0.06153132, -0.3575046 , -0.6215774 ,
        2.5407457 , -0.5688942 ,  0.6047784 ,  1.4386171 ,  0.84

In [38]:
#t = tokenizeFile(1)
inferred_vector = model.infer_vector( t )
# dv is short for docvecs
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
#sims = model.dv.most_similar([inferred_vector], topn=4)
sims[10]

(147, 0.6333470344543457)

In [39]:
# letter with filename 1.txt is the first letter, so tag is 0
similar_doc = model.docvecs.most_similar(1)
similar_doc

<ipython-input-39-e11ffd9e85f7>:2: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  similar_doc = model.docvecs.most_similar(1)


[(171, 0.804776132106781),
 (91, 0.787969172000885),
 (299, 0.7870095372200012),
 (112, 0.7683703899383545),
 (130, 0.7682340145111084),
 (271, 0.7531971335411072),
 (222, 0.7139574885368347),
 (267, 0.6966142058372498),
 (29, 0.6955546736717224),
 (119, 0.6954190135002136)]

In [40]:
print('number of documents', model.corpus_count)
print('model.docvecs', len(model.dv))

number of documents 300
model.docvecs 300


In [41]:
# Hipergator
# reread the files, and get the vector for each file
# feed vector into k-means algorithm to make clusters
wordLists = MyCommentLetters(r'/blue/acg7849/share/BS/item1/', tokens_only = True) # a memory-friendly iterator
vectors = [ model.infer_vector( w ) for w in wordLists]
len(vectors)

300

In [42]:
vectors[0]

array([-1.1704612 , -5.130845  , -3.15631   ,  0.44030204, -0.4878984 ,
       -1.1170868 , -3.2990344 , -4.6746206 , -5.4853024 , -2.2243352 ,
        1.3230593 ,  2.7618692 , -4.4890447 ,  4.0193677 , -3.94044   ,
       -0.39660156,  1.2420162 , -0.41996512, -3.2657363 , -2.8233726 ,
       -1.0030143 , -0.5569476 ,  0.93863446, -0.45166194,  4.879965  ,
       -0.9073644 ,  2.0911472 ,  3.250878  , -0.93150634,  0.268633  ,
       -2.8722794 ,  1.7253896 , -2.902139  , -0.9727945 ,  1.1975088 ,
       -2.748817  ,  1.5173533 ,  0.6490475 , -0.525698  ,  0.6934823 ,
       -2.347691  ,  1.1805925 ,  0.6909716 , -1.3081062 ,  0.65291697,
        0.6771697 , -2.9531481 ,  1.1721892 ,  1.6826416 , -0.87920207,
       -0.44763726, -0.20184833, -2.0522463 , -3.5810897 , -0.04629781,
        2.9264932 ,  0.4587552 ,  2.249673  ,  2.5155492 ,  2.9339454 ,
       -1.612512  ,  1.4867091 ,  0.24880655, -0.44831407, -0.5424832 ,
        2.1672578 , -0.496822  ,  0.53103   ,  1.5260354 ,  0.79

In [44]:
import nltk
from nltk.cluster import KMeansClusterer
num_clusters = 50
kclusterer = KMeansClusterer(num_clusters, distance=nltk.cluster.util.cosine_distance, repeats=25)
assigned_clusters = kclusterer.cluster(vectors, assign_clusters=True)

In [51]:
assigned_clusters[0:20]

[9, 46, 33, 18, 49, 25, 34, 28, 29, 46, 42, 1, 24, 39, 14, 44, 41, 12, 32, 21]

In [52]:
import collections

print(collections.Counter(assigned_clusters))

Counter({27: 22, 17: 17, 43: 17, 41: 15, 21: 12, 42: 9, 47: 9, 46: 8, 34: 8, 39: 8, 40: 8, 35: 8, 6: 8, 28: 7, 10: 7, 20: 7, 33: 6, 1: 6, 16: 6, 3: 6, 45: 6, 48: 6, 18: 5, 49: 5, 14: 5, 44: 5, 31: 5, 13: 5, 19: 5, 15: 5, 38: 5, 9: 4, 29: 4, 12: 4, 2: 4, 4: 4, 25: 3, 24: 3, 32: 3, 0: 3, 11: 3, 37: 3, 23: 2, 22: 2, 7: 2, 36: 1, 26: 1, 30: 1, 8: 1, 5: 1})


## In-class question

For each cluster, what are the 20 most frequent words (excluding stop words, excluding punctuation)


In [ ]:
# IndexError: list index out of range  (with 300 files read)

#keep track of fileIds for letters in different clusters
ids = [ [] for i in range(10) ]
for i, clust in enumerate(assigned_clusters):
    # look up the filename for i (which is the tag), then add it to the list for that cluster
    ids[ clust - 1 ].append( indexToFileId[ i ]  )     

In [24]:
# first two clusters
ids[0:2]

[[266806,
  276013,
  278199,
  285320,
  275923,
  284230,
  278425,
  275820,
  281337,
  268181,
  279912,
  275007],
 [285179,
  280499,
  281703,
  272933,
  265514,
  266283,
  271008,
  265393,
  277411,
  271174,
  283258,
  277446,
  269522,
  271690,
  269881,
  274401,
  279101,
  279597]]

In [26]:
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.collocations import *
# now read the comment letters for each cluster

In [27]:
for i, clust in enumerate(ids):
    content = ""
    for file in clust:
        with open( str(file) + ".txt", encoding='utf-8') as f:# No such file or directory: '266806.txt'
            content += f.read()
    file_tokens = [x.lower() for x in word_tokenize(content) if x.isalpha() and x.lower() not in stopWords and x not in string.punctuation]
    # now we can use nltk functions on the text
    fdist = FreqDist(file_tokens)
    print ('cluster', i+ 1, 'most common words in this letter', fdist.most_common(5) )

FileNotFoundError: [Errno 2] No such file or directory: '266806.txt'